# Genera el XML de las personas

## Resumen del Código para Generar el Archivo XML con LBPH
Reconocedor de rostros.a:

1. **Importación de Librerías:**
   - `cv2` para el procesamiento de imágenes.
   - `numpy` para manejo de arreglos.
   - `os` para manejo del sistema de archivos.

2. **Definición del Conjunto de Datos:**
   - Se especifica la ruta del directorio con subdirectorios que contienen imágenes etiquetadas de rostros.

3. **Lectura y Procesamiento de Datos:**
   - Se obtiene una lista de los nombres de los subdirectorios (cada uno representando una persona).
   - Se inicializan listas para almacenar las etiquetas y los datos de las imágenes.
   - Para cada subdirectorio, se leen las imágenes en escala de grises y se almacenan junto con sus etiquetas correspondientes.

4. **Entrenamiento del Modelo:**
   - Se crea un reconocedor facial LBPH.
   - Se entrena el reconocedor con las imágenes (`facesData`) y sus etiquetas (`labels`).

5. **Guardado del Modelo:**
   - El modelo entrenado se guarda en un archivo XML (`LBPHPersonesFix.xml`) para su uso posterior.


In [ ]:
import cv2 as cv 
import numpy as np 
import os

# Ruta del conjunto de datos que contiene las imágenes etiquetadas de los rostros
dataSet = 'D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Rostros\\Persones'

# Obtener los nombres de los directorios dentro del conjunto de datos
faces = os.listdir(dataSet)
print(faces)

# Inicializar listas para las etiquetas y los datos de las caras
labels = []
facesData = []
label = 0 

# Iterar sobre cada directorio en el conjunto de datos
for face in faces:
    facePath = dataSet + '\\' + face  # Ruta del directorio de cada persona
    for faceName in os.listdir(facePath):
        labels.append(label)  # Agregar la etiqueta correspondiente a la lista de etiquetas
        # Leer la imagen en escala de grises y agregarla a la lista de datos de caras
        facesData.append(cv.imread(facePath + '\\' + faceName, 0))
    label = label + 1  # Incrementar la etiqueta para la siguiente persona

# Crear un reconocedor facial LBPH
faceRecognizer = cv.face.LBPHFaceRecognizer_create()

# Entrenar el reconocedor con los datos de las caras y sus etiquetas
faceRecognizer.train(facesData, np.array(labels))

# Guardar el modelo entrenado en un archivo XML
faceRecognizer.write('D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Rostros\\LBPHPersonesFix.xml')

# Corre el XML 

## Resumen del Código para Reconocimiento Facial en Tiempo Reala:

1. **Importación de Librerías:**
   - `cv2` para procesamiento de imágenes.
   - `os` para manejo de archivos del sistema.

2. **Carga del Modelo:**
   - Se crea un reconocedor LBPH y se carga un modelo preentrenado desde un archivo XML.

3. **Preparación del Conjunto de Datos:**
   - Se especifica la ruta del directorio que contiene las imágenes etiquetadas de rostros.

4. **Inicialización de la Cámara:**
   - Se inicia la captura de video desde la cámara web.
   - Se carga un clasificador Haar para la detección de rostros.

5. **Procesamiento en Tiempo Real:**
   - Se captura y procesa cada fotograma de la cámara.
   - Se convierte el fotograma a escala de grises y se detectan rostros.

6. **Reconocimiento Facial:**
   - Se extrae, redimensiona y predice la región de interés de cada rostro detectado.
   - Se muestran los resultados en el fotograma, etiquetando los rostros como conocidos o desconocidos.

7. **Visualización y Salida:**
   - Se muestra el fotograma con los resultados en una ventana.
   - Se espera la tecla 'Esc' para finalizar el programa.

8. **Liberación de Recursos:**
   - Se liberan los recursos de la cámara y se cierran las ventanas de OpenCV.


In [2]:
import cv2 as cv
import os 

# Crear el reconocedor facial LBPH y cargar el modelo preentrenado desde un archivo XML
faceRecognizer = cv.face.LBPHFaceRecognizer_create()
faceRecognizer.read('D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Rostros\\LBPHPersonesFix.xml')

# Definir la ruta del conjunto de datos que contiene las etiquetas de las caras
dataSet = 'D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Rostros\\Persones'
faces = os.listdir(dataSet)  # Obtener los nombres de los archivos en el directorio

# Iniciar la captura de video desde la cámara web
cap = cv.VideoCapture(0)

# Cargar el clasificador Haar para la detección de rostros
rostro = cv.CascadeClassifier('D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Material de clase\\haarcascade_frontalface_alt.xml')

while True:
    # Leer un fotograma de la cámara
    ret, frame = cap.read()
    if ret == False: 
        break  # Salir del bucle si no se puede leer el fotograma

    # Convertir el fotograma a escala de grises
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()  # Hacer una copia de la imagen en grises

    # Detectar rostros en la imagen en escala de grises
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    
    for (x, y, w, h) in rostros:
        # Extraer la región de interés (ROI) que contiene el rostro detectado
        frame2 = cpGray[y:y+h, x:x+w]
        
        # Redimensionar la ROI a 100x100 píxeles
        frame2 = cv.resize(frame2, (100, 100), interpolation=cv.INTER_CUBIC)
        
        # Realizar la predicción de la cara utilizando el modelo LBPH
        result = faceRecognizer.predict(frame2)
        
        # Mostrar el resultado de la predicción en el fotograma
        cv.putText(frame, '{}'.format(result), (x, y-20), 1, 3.3, (255, 255, 0), 1, cv.LINE_AA)
        
        if result[1] < 100:
            # Si la confianza es menor a 100, se considera una coincidencia y se muestra el nombre
            cv.putText(frame, '{}'.format(faces[result[0]]), (x, y-25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            # Si la confianza es mayor o igual a 100, se considera "Desconocido"
            cv.putText(frame, 'Desconocido', (x, y-20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Mostrar el fotograma con los resultados en una ventana
    cv.imshow('frame', frame)
    
    # Esperar 1 milisegundo por una tecla, si es 'Esc' (código 27), salir del bucle
    k = cv.waitKey(1)
    if k == 27:
        break

# Liberar la cámara y cerrar todas las ventanas de OpenCV
cap.release()
cv.destroyAllWindows()

# Genera el XML de Emociones

# Resumen del Código
:

1. **Importación de Librerías:**
   - `cv2` para procesamiento de imágenes.
   - `numpy` para manejo de arreglos.
   - `os` para manejo del sistema de archivos.

2. **Definición del Conjunto de Datos:**
   - Se especifica la ruta del directorio con imágenes etiquetadas de rostros.

3. **Lectura y Procesamiento de Datos:**
   - Se obtiene una lista de los nombres de los subdirectorios (cada uno representando una persona).
   - Se inicializan listas para almacenar las etiquetas y los datos de las imágenes.
   - Para cada subdirectorio, se leen las imágenes en escala de grises y se almacenan junto con sus etiquetas correspondientes.

4. **Entrenamiento del Modelo:**
   - Se crea un reconocedor facial LBPH.
   - Se entrena el reconocedor con las imágenes y sus etiquetas.

5. **Guardado del Modelo:**
   - El modelo entrenado se guarda en un archivo XML para su uso posterior.


In [ ]:
import cv2 as cv 
import numpy as np 
import os

# Ruta del conjunto de datos que contiene las imágenes etiquetadas de los rostros
dataSet = 'D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Emociones\\Emociones3'

# Obtener los nombres de los directorios dentro del conjunto de datos
faces  = os.listdir(dataSet)
print(faces)

# Inicializar listas para las etiquetas y los datos de las caras
labels = []
facesData = []
label = 0 

# Iterar sobre cada directorio en el conjunto de datos
for face in faces:
    facePath = dataSet + '\\' + face  # Ruta del directorio de cada persona
    for faceName in os.listdir(facePath):
        labels.append(label)  # Agregar la etiqueta correspondiente a la lista de etiquetas
        # Leer la imagen en escala de grises y agregarla a la lista de datos de caras
        facesData.append(cv.imread(facePath + '\\' + faceName, 0))
    label = label + 1  # Incrementar la etiqueta para la siguiente persona

# Crear un reconocedor facial LBPH
faceRecognizer = cv.face.LBPHFaceRecognizer_create()

# Entrenar el reconocedor con los datos de las caras y sus etiquetas
faceRecognizer.train(facesData, np.array(labels))

# Guardar el modelo entrenado en un archivo XML
faceRecognizer.write('D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Emociones\\EmocionesLBPHZwei.xml')


# Correr XML

Aqui se corre el xml junto con el haarcascade para correr  el xml de las emociones.

## Resumen del Códigoa:

1. **Importación de Librerías:**
   - `cv2` para visión por computadora.
   - `os` para manejo de archivos del sistema.

2. **Carga del Modelo:**
   - Se crea un reconocedor LBPH y se carga un modelo preentrenado desde un archivo XML.

3. **Preparación del Conjunto de Datos:**
   - Se especifica la ruta del directorio que contiene imágenes etiquetadas de rostros.

4. **Inicialización de la Cámara:**
   - Se inicia la captura de video desde la cámara web.
   - Se carga un clasificador Haar para la detección de rostros.

5. **Procesamiento en Tiempo Real:**
   - Se captura cada fotograma de la cámara.
   - Se convierte el fotograma a escala de grises.
   - Se detectan rostros en la imagen en escala de grises.

6. **Reconocimiento Facial:**
   - Para cada rostro detectado, se extrae y redimensiona la región de interés.
   - Se realiza la predicción utilizando el modelo LBPH.
   - Se muestra el resultado en el fotograma:
     - Si la confianza es menor a 100, se muestra el nombre y un rectángulo verde.
     - Si la confianza es mayor o igual a 100, se etiqueta como "Desconocido" y se dibuja un rectángulo rojo.

7. **Visualización y Salida:**
   - Se muestra el fotograma con los resultados en una ventana.
   - Se espera la tecla 'Esc' para salir del bucle y finalizar el programa.

8. **Liberación de Recursos:**
   - Se liberan los recursos de la cámara y se cierran las ventanas de OpenCV.


In [1]:
import cv2 as cv
import os 

# Crear el reconocedor facial LBPH y cargar el modelo preentrenado
faceRecognizer = cv.face.LBPHFaceRecognizer_create()
faceRecognizer.read('D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Emociones\\EmocionesLBPHZwei.xml')

# Definir la ruta del conjunto de datos que contiene las etiquetas de las caras
dataSet = 'D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Emociones\\Emociones3'
faces = os.listdir(dataSet)  # Obtener los nombres de los archivos en el directorio

# Iniciar la captura de video desde la cámara web
cap = cv.VideoCapture(0)

# Cargar el clasificador Haar para la detección de rostros
rostro = cv.CascadeClassifier('D:\\Escuela Betin\\9no Sem\\3. Inteligencia Artificial\\Material de clase\\haarcascade_frontalface_alt.xml')

while True:
    # Leer un fotograma de la cámara
    ret, frame = cap.read()
    if ret == False: 
        break  # Salir del bucle si no se puede leer el fotograma

    # Convertir el fotograma a escala de grises
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    cpGray = gray.copy()  # Hacer una copia de la imagen en grises

    # Detectar rostros en la imagen en escala de grises
    rostros = rostro.detectMultiScale(gray, 1.3, 3)
    
    for (x, y, w, h) in rostros:
        # Extraer la región de interés (ROI) que contiene el rostro detectado
        frame2 = cpGray[y:y+h, x:x+w]
        
        # Redimensionar la ROI a 100x100 píxeles
        frame2 = cv.resize(frame2,  (100, 100), interpolation=cv.INTER_CUBIC)
        
        # Realizar la predicción de la cara utilizando el modelo LBPH
        result = faceRecognizer.predict(frame2)
        
        # Mostrar el resultado de la predicción en el fotograma
        cv.putText(frame, '{}'.format(result), (x, y-20), 1, 3.3, (255, 255, 0), 1, cv.LINE_AA)
        
        if result[1] < 100:
            # Si la confianza es menor a 100, se considera una coincidencia y se muestra el nombre
            cv.putText(frame, '{}'.format(faces[result[0]]), (x, y-25), 2, 1.1, (0, 255, 0), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        else:
            # Si la confianza es mayor o igual a 100, se considera "Desconocido"
            cv.putText(frame, 'Desconocido', (x, y-20), 2, 0.8, (0, 0, 255), 1, cv.LINE_AA)
            cv.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

    # Mostrar el fotograma con los resultados en una ventana
    cv.imshow('frame', frame)
    
    # Esperar 1 milisegundo por una tecla, si es 'Esc' (código 27), salir del bucle
    k = cv.waitKey(1)
    if k == 27:
        break

# Liberar la cámara y cerrar todas las ventanas de OpenCV
cap.release()
cv.destroyAllWindows()